db structure:
```mermaid
erDiagram
    Match {
        string match_id PK
        datetime match_date
        string home_team_id FK
        string away_team_id FK
        int home_score
        int away_score
    }
    
    Team {
        string team_id PK
        string team_name
    }
    
    Player {
        string player_id PK
        string player_name
        string team_id FK
        int jersey_number
    }
    
    PlayerTracking {
        int id PK
        string game_id FK
        bigint frame_id
        string timestamp
        int period_id
        string player_id FK
        float x
        float y
    }
    
    EventType {
        string eventtype_id PK
        string name
        string description
    }
    
    QualifierType {
        string qualifier_id PK
        string name
        string description
    }
    
    MatchEvent {
        string match_id PK, FK
        string event_id PK
        string eventtype_id FK
        string result
        boolean success
        int period_id
        string timestamp
        string end_timestamp
        string ball_state
        string ball_owning_team
        string team_id FK
        string player_id FK
        float x
        float y
        float end_coordinates_x
        float end_coordinates_y
        string receiver_player_id FK
    }
    
    Qualifier {
        string match_id PK, FK
        string event_id PK, FK
        string qualifier_type_id PK, FK
        string qualifier_value
    }
    
    SpadlAction {
        int id PK
        string game_id FK
        int period_id
        float seconds
        string player_id FK
        string team_id FK
        float start_x
        float start_y
        float end_x
        float end_y
        string action_type
        string result
        string bodypart
    }
    
    Match ||--o{ PlayerTracking : "has"
    Match ||--o{ MatchEvent : "has"
    Match ||--o{ SpadlAction : "has"
    Team ||--o{ Player : "has"
    Team ||--o{ MatchEvent : "participates in"
    Team ||--o{ SpadlAction : "performs"
    Player ||--o{ PlayerTracking : "has"
    Player ||--o{ MatchEvent : "performs"
    Player ||--o{ MatchEvent : "receives"
    Player ||--o{ SpadlAction : "performs"
    EventType ||--o{ MatchEvent : "categorizes"
    MatchEvent ||--o{ Qualifier : "has"
    QualifierType ||--o{ Qualifier : "categorizes"
    Team ||--o{ Match : "home team"
    Team ||--o{ Match : "away team"
```

# Setup and Connection
Import the required libraries and establish database connection using environment variables from a .env file.

In [ ]:
# Import necessary libraries
import pandas as pd
import numpy as np
import psycopg2
import os
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv()

def get_database_connection():
    """Create and return a database connection using environment variables."""
    return psycopg2.connect(
        host=os.getenv("PG_HOST"),
        database=os.getenv("PG_DB"),
        user=os.getenv("PG_USER"),
        password=os.getenv("PG_PASSWORD"),
        port=os.getenv("PG_PORT")
    )

# Team Selection
Use find_team_id() function to search for and select a team to analyze based on user input.

In [ ]:
# Team Selection

# Prompt the user to input a team name
team_name = input("Enter the team name to search for: ")

# Use the find_team_id() function to search for the team
def find_team_id(team_name):
    """Find the team ID for a given team name, using case-insensitive partial matching."""
    conn = get_database_connection()
    try:
        with conn.cursor() as cursor:
            # Use ILIKE for case-insensitive partial matching
            cursor.execute("SELECT team_id, team_name FROM teams WHERE team_name ILIKE %s", (f'%{team_name}%',))
            results = cursor.fetchall()
            
            if results:
                print(f"Found {len(results)} teams matching '{team_name}':")
                for team_id, full_name in results:
                    print(f"  - {full_name} (ID: {team_id})")
                
                # Return the first match
                return results[0][0], results[0][1]
            else:
                print(f"No team found matching '{team_name}'")
                return None, None
    finally:
        conn.close()

# Call the function and display the selected team
team_id, full_team_name = find_team_id(team_name)
if team_id:
    print(f"Selected Team: {full_team_name} (ID: {team_id})")
else:
    print("No team selected.")

# Match Selection
Use find_matches_with_team() function to identify available matches for the selected team, and allow the user to choose a specific match.

In [ ]:
# Match Selection

# Use the find_matches_with_team() function to identify available matches for the selected team
def find_matches_with_team(team_id):
    """Find all matches containing a specific team."""
    conn = get_database_connection()
    try:
        with conn.cursor() as cursor:
            query = """
            SELECT DISTINCT pt.game_id
            FROM player_tracking pt
            JOIN players p ON pt.player_id = p.player_id
            WHERE p.team_id = %s
            """
            cursor.execute(query, (team_id,))
            matches = [row[0] for row in cursor.fetchall()]
            
            if matches:
                print(f"Found {len(matches)} matches with this team:")
                for i, match_id in enumerate(matches[:5], 1):
                    print(f"  {i}. {match_id}")
                
                if len(matches) > 5:
                    print(f"  ...and {len(matches)-5} more")
            else:
                print("No matches found with this team")
            
            return matches
    finally:
        conn.close()

# Ensure a team is selected before proceeding
if team_id:
    # Fetch available matches for the selected team
    available_matches = find_matches_with_team(team_id)
    
    # Allow the user to select a specific match
    if available_matches:
        print("\nAvailable Matches:")
        for idx, match_id in enumerate(available_matches, 1):
            print(f"{idx}. {match_id}")
        
        match_index = input("\nEnter the number of the match you want to analyze: ")
        try:
            match_index = int(match_index) - 1
            if 0 <= match_index < len(available_matches):
                selected_match_id = available_matches[match_index]
                print(f"Selected Match ID: {selected_match_id}")
            else:
                print("Invalid selection. No match selected.")
                selected_match_id = None
        except ValueError:
            print("Invalid input. No match selected.")
            selected_match_id = None
    else:
        selected_match_id = None
else:
    selected_match_id = None

# Data Extraction
Extract player tracking data for the selected team and match using get_team_tracking_data() function.

In [ ]:
# Data Extraction

# Ensure a team and match are selected before proceeding
if team_id and selected_match_id:
    # Define the function to extract tracking data
    def get_team_tracking_data(team_id, match_id):
        """
        Extract tracking data for a specific team and match from the database.
        
        Args:
            team_id: ID of the team to analyze
            match_id: ID of the match to analyze
        
        Returns:
            DataFrame with tracking data
        """
        conn = get_database_connection()
        try:
            with conn.cursor() as cursor:
                # Get player IDs for the team
                cursor.execute("SELECT player_id FROM players WHERE team_id = %s", (team_id,))
                player_ids = [row[0] for row in cursor.fetchall()]
                
                if not player_ids:
                    print("No players found for this team")
                    return pd.DataFrame()
                    
                # Format the list for SQL IN clause
                player_ids_str = "'" + "','".join(str(id) for id in player_ids) + "'"
                
                # Build the query
                query = f"""
                SELECT pt.*, p.player_name, p.jersey_number, t.team_name
                FROM player_tracking pt
                JOIN players p ON pt.player_id = p.player_id
                JOIN teams t ON p.team_id = t.team_id
                WHERE p.player_id IN ({player_ids_str}) AND pt.game_id = '{match_id}'
                ORDER BY pt.frame_id, pt.player_id
                """
                
                print(f"Executing query to fetch tracking data for match {match_id}...")
                cursor.execute(query)
                columns = [desc[0] for desc in cursor.description]
                data = cursor.fetchall()
                
                # Create DataFrame
                df = pd.DataFrame(data, columns=columns)
                return df
        finally:
            conn.close()
    
    # Fetch tracking data for the selected team and match
    tracking_data = get_team_tracking_data(team_id, selected_match_id)
    
    # Display the first few rows of the tracking data
    if not tracking_data.empty:
        print(f"Tracking data for match {selected_match_id} loaded successfully.")
        display(tracking_data.head())
    else:
        print("No tracking data found for the selected match.")
else:
    print("Team or match not selected. Cannot extract tracking data.")

# Defensive Pattern Analysis
Calculate defensive metrics and identify patterns using calculate_defensive_metrics() and identify_defensive_patterns() functions.

In [ ]:
# Function to check if a match has player data (not just the ball)
def find_match_with_player_data():
    conn = get_database_connection()
    try:
        with conn.cursor() as cursor:
            # Get your team ID
            team_id, full_team_name = find_team_id("Club Brugge")
            
            # Get all matches for this team
            matches = find_matches_with_team(team_id)
            
            # Check each match for player data
            for match_id in matches:
                print(f"Checking match {match_id}...")
                
                # Get a sample of data
                query = f"""
                SELECT player_id, COUNT(*) as count
                FROM player_tracking pt
                WHERE game_id = '{match_id}'
                GROUP BY player_id
                """
                cursor.execute(query)
                player_counts = pd.DataFrame(cursor.fetchall(), columns=['player_id', 'count'])
                
                # Count non-ball tracking points
                non_ball_data = player_counts[player_counts['player_id'] != 'ball']
                
                if len(non_ball_data) > 0:
                    print(f"✓ FOUND PLAYER DATA: Match {match_id} has data for {len(non_ball_data)} players")
                    return match_id
                else:
                    print(f"✗ Match {match_id} only has ball data")
                    
            print("No matches found with player tracking data")
            return None
    finally:
        conn.close()

# Find a match with player data
match_with_players = find_match_with_player_data()

if match_with_players:
    print(f"\nFound a match with player data: {match_with_players}")
    print("Let's use this match for analysis")
    
    # Now get full tracking data for this match
    tracking_data = get_team_tracking_data(team_id, match_with_players)
    
    # Display a summary
    player_counts = tracking_data.groupby('frame_id').size()
    print(f"Player count statistics:")
    print(f"Min players per frame: {player_counts.min()}")
    print(f"Max players per frame: {player_counts.max()}")
    print(f"Mean players per frame: {player_counts.mean():.2f}")
    
    # Try the analysis again with this new data
    defensive_metrics = calculate_defensive_metrics_debug(tracking_data)
else:
    print("Could not find any matches with player tracking data")

# Visualization of Results
Visualize the identified defensive patterns using plot_pattern_examples() and analyze pattern usage with analyze_pattern_usage() function.

# Animation of Defensive Movements
Create an animation showing how defensive patterns change over time using create_defensive_animation() function.

In [ ]:
pip install socceraction[statsbomb]